In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.image as image
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn import preprocessing
from random import shuffle
from numpy import argmax

# **Reading dataset**

In [2]:

train = glob.glob("../input/chess-positions/train/*.jpeg")
test = glob.glob("../input/chess-positions/test/*.jpeg")


In [3]:
train_size = 3000
test_size = 1000
shuffle(train)
shuffle(test)
train = train[:train_size]
test = test[:test_size]


In [4]:
def fen_label_extraction(data):
    labels = []
    for img in data:
        base = os.path.basename(img)
        labels.append(os.path.splitext(base)[0])
    return labels
    
def getX(data):
    X = [cv2.imread(i) for i in data]
    return X
    
def rgb_to_gray(X):
    X_out=[]
    for i in X:
        image_grey = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
        X_out.append(image_grey)
    return X_out

def split_chess_board(board):
    temp = []
    for x in range(0,8):
        temp2 = board[(x*25):((x+1)*25),:]
        for y in range(0,8):
            temp.append(temp2[:,(y*25):((y+1)*25)].flatten().reshape(25,25,3))
    return temp

def split_labels(y_train):
    labels = []
    for i in y_train:
        for ches_row in i.split('-'):
            for item in ches_row:
                if(item.isdigit()):
                    empty_pos = int(item)
                    for i in range(empty_pos):
                        labels.append('1')
                else:
                    labels.append(item)
    return labels

def prepare_test_image(img):
    test_img = cv2.imread(img)
    board  =cv2.resize(test_img, (200, 200))
    temp = []
    for x in range(0,8):
        temp2 = board[(x*25):((x+1)*25),:]
        for y in range(0,8):
            temp.append(temp2[:,(y*25):((y+1)*25)].flatten().reshape(25,25,3))
            
    tes = np.array(temp).astype("float32") / 255
    return tes,test_img

def preprocess(X):
    temp=[]
    for i in X:
        img  =cv2.resize(i, (200, 200))
        temp.extend(split_chess_board(img))  
    return temp

def decode_fen(prediction):
    predicted_fen = []
    for i in prediction[0]:
        row = le.inverse_transform(i)
        fen = ''
        count = 0
        digit_flag = False
        for j in range(len(row)):
            if row[j].isdigit():
                count = count+1
                digit_flag = True
                if(j==7):
                    fen = fen+ str(count)
            else:
                if digit_flag:
                    fen = fen+ str(count)
                    fen = fen+row[j]
                else:
                    fen = fen+row[j]
                digit_flag = False
                count = 0
        predicted_fen.append(fen)
    return '-'.join(predicted_fen)



In [5]:
y_train_global = np.array(fen_label_extraction(train))
y_test_global = np.array(fen_label_extraction(test))
X_train_global = np.array(getX(train))
X_test_global = np.array(getX(test))

In [6]:
y_train = y_train_global
y_test = y_test_global
X_train = X_train_global
X_test = X_test_global

# **Visualize Data**

In [7]:
fig, ax = plt.subplots(1,4, figsize=(15,15))
for i in range(4):
    ax[i].set_title(y_train[i])
    ax[i].imshow(X_train[i])

# **Preprocessing Train data**

In [8]:
X_train = preprocess(X_train)
fig, ax = plt.subplots(1,8, figsize=(15,15))
for i in range(8):
    ax[i].imshow(X_train[i])


# **Preprocessing train labels**

In [9]:
y = np.array(split_labels(y_train), dtype='str')
print(y_train[0])
print(y[0],y[1],y[2],y[3],y[4],y[5],y[6],y[7])

In [10]:
print("Length of training data: ",len(X_train))
print("Length of training labels: ",len(y))

In [11]:
unique_labels = list(set(y))
print('unique_labels: ',unique_labels)

# **Label encoder for train**

In [12]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y)
print('Encoded classes: ',list(le.classes_))
print(y_train[:7])
print(y[:7])


# **Prepocessing Test Data**

In [13]:
X_test = preprocess(X_test)

In [14]:
from keras.utils import np_utils
y_t = np.array(split_labels(y_test),dtype='str')
y_test = le.transform(y_t)


In [15]:
plt.imshow(X_test[3])
print('y_test:',y_test[3])

In [16]:
X_train = (np.array(X_train))
y_train = (np.array(y_train))
X_train = X_train.astype("float32") / 255

print("x_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_test = (np.array(X_test))
y_test = (np.array(y_test))
X_test = X_test.astype("float32") / 255

print("x_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)




# **Designing Neural Net**

In [17]:
model = Sequential()
model.add(Conv2D(25, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(25, 25, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(50, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(13, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
model.summary()


In [18]:
history = model.fit(X_train,y_train, epochs = 2, validation_data=(X_test, y_test))


# **Make Prediction**

In [27]:
from random import randint

random_index = randint(0,999)
test_image,test_img = prepare_test_image(test[random_index])


In [28]:
res = (
  model.predict(test_image)
  .argmax(axis=1)
  .reshape(-1, 8, 8)
)
res

In [29]:
plt.imshow(test_img)


In [31]:
print("Actual label:", test[random_index].split('/')[-1] )
print("predicted FEN:",decode_fen(res))
